1. Cài đặt thư viện cần thiết

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torch torchaudio tqdm ffmpeg transformers datasets peft accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.6 MB/s eta

4. Import thư viện cần thiết

In [ ]:
import os
import torch
import whisper
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model
from torch.optim import AdamW
from peft import PeftModel

5. Kiểm tra thiết bị

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔹 Using device:", device)

🔹 Using device: cuda


In [ ]:
import os
import torch
import whisper
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model
from torch.optim import AdamW
from peft import PeftModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔹 Using device:", device)

# 1) Load mô hình Whisper gốc (cùng phiên bản bạn đã dùng khi fine-tune)
base_model_name = "openai/whisper-small"  # hoặc whisper-medium, large...
base_model = WhisperForConditionalGeneration.from_pretrained(base_model_name)
base_model.to(device)
base_model.eval()

# 2) Nạp trọng số và config LoRA từ thư mục tmp_model_dir
lora_model_dir = "/content/models"  # đường dẫn chứa adapter_config.json, adapter_model.safetensors
model = PeftModel.from_pretrained(base_model, lora_model_dir)
model.to(device)
model.eval()

# 3) Nạp processor (hoặc load lại processor cũ của bạn)
processor = WhisperProcessor.from_pretrained(base_model_name)

# 4) Hàm suy luận (transcribe) cho 1 file audio
def transcribe_audio(audio_path):
    # Load audio bằng whisper.load_audio (có sẵn của openai/whisper)
    audio_data = whisper.load_audio(audio_path)
    # Xử lý thành input features
    inputs = processor(audio_data, return_tensors="pt", sampling_rate=16000)
    input_features = inputs.input_features.to(device)

    # Gọi model.generate để lấy token dự đoán
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # Decode token IDs về text
    transcription = processor.decode(predicted_ids[0])
    return transcription

🔹 Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
# 5) Ví dụ suy luận
audio_file = "/content/Bản ghi Mới 7.m4a"
predicted_text = transcribe_audio(audio_file)
print("Kết quả nhận dạng:", predicted_text)

Kết quả nhận dạng:  ಇ Ngày mười lăm tháng ba hai không hai lăm nhân dịệp ông nhậm chức thủ tướng thương hai mười tư của ca na đa thủ tướng chính phủ và minh chín đã gửi thư chúc mừng
